# Creating a chord progression with a genetic algorithm

This work is the result of an experiment done some months ago. I used a simple genetic algorithm to find a solution to a classic exercise of harmony: given a certain voice (normally the bass) create the other three voices to make a chord progression. The code isn't perfect and the algorithm can be improved adding more options in the chord selection, but for simplicity I didn't use seventh chords.

In [1]:
# Casos de prueba:
# 2# 54 67 69 61 62
# 1# 43 50 48 47 45 43 55 52 50 48 52 50 50 43 45 47 48 47 48 52 50 38 43
# 2# 50 43 49 50 52 45 49 50 43 45 50
# 2b 46 50 48 51 53 51 48 45 46

## Working with music

The first part of this challenge is to find a way to easily represent the notes in the melody, luckily for us many years ago MIDI was created, so I used the numbers in MIDI to match every single note being, for example, 60 the central C. And at the beginning of the sequence of notes I set the key signature with the number of sharps or flats.
So an example like this:

In [2]:
import random
import math
import numpy

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [3]:
# Global Variables
OPTIONS_M = ((0,-3,5), (0,-3,5), (0,-4,5), (0,-3,6), (0,-3,5), (0,-4,5), (0,-4,5))
OPTIONS_m = ((0,-4,5), (0,-4,5), (0,-3,5), (0,-3,5), (0,-4,5), (0,-3,6), (0,5))
MOD_M = ('M','m','m','M','M','m','d')
MOD_m = ('m','d','M','m','M','M','M')

In [4]:
def neighborhood(iterable):
    """Generator gives the prev actual and next."""
    iterator = iter(iterable)
    prev = None
    item = next(iterator)  # throws StopIteration if empty.
    for nex in iterator:
        yield (prev,item,nex)
        prev = item
        item = nex
    yield (prev,item,None)

In [5]:
def setTon(line):
    """Determine the tonality of the exercice."""
    ton = line[:2]
    notes = list(map(int, line[3:].split(' ')))
    if ton[1] == '#':
        ton = (int(ton[0])*7)%12
    else:
        ton = (int(ton[0])*5)%12
    for note in notes:
        if (ton+6)%12 == note%12:
            ton = str((ton-3)%12)+'m'
            break
    else:
        if ton-3 == notes[-1]%12:
            ton = str((ton-3)%12)+'m'
        else:
            ton = str(ton)+'M'
    return ton, notes

In [6]:
def creatChord(nameC, noteF):
    """Create one chord given the name of the chord and the fundamental note."""
    num_funda = int(nameC[:-1])
    if nameC[-1] == 'M':
        val_notes = [num_funda, (num_funda+4)%12, (num_funda+7)%12]
    elif nameC[-1] == 'm':
        val_notes = [num_funda, (num_funda+3)%12, (num_funda+7)%12]
    elif nameC[-1] == 'd':
        val_notes = [num_funda, (num_funda+3)%12, (num_funda+6)%12]
        
    tenorR = list(range(48, 69))
    contR = list(range(52, 77))
    sopR = list(range(60, 86))
    
    # Depending in the bass note this are the options for the others voices
    if noteF%12 == val_notes[0]:
        opc = [[1,1,1], [2,1,0], [0,1,2]]
    elif noteF%12 == val_notes[1]:
        opc = [[1,0,2], [3,0,0], [2,0,1]]
    elif noteF%12 == val_notes[2]:
        opc = [[1,1,1], [2,1,0]]
    
    opc = random.choice(opc)
    chordN = list()
    for num, val in zip(opc, val_notes):
        chordN += [val]*num
    
    random.shuffle(chordN)
    
    chord = [noteF,]
    for nte, voce in zip(chordN, [tenorR, contR, sopR]):
        posible_n = [x for x in voce if x%12 == nte]
        chord.append(random.choice(posible_n))
    
    return chord

In [7]:
def selChord(ton, notesBass):
    """Select the chords from all the posibilities."""
    listaOp = OPTIONS_M if ton[-1] == 'M' else OPTIONS_m
    listaMod = MOD_M if ton[-1] == 'M' else MOD_m
    prog = list()
    
    for note in notesBass:
        name = note%12
        grad = name-int(ton[:-1])
        grad = math.ceil(((grad+12)%12) / 2)
        num = (listaOp[grad][random.randint(0,len(listaOp[grad])-1)] + name +12) %12
        grad = num-int(ton[:-1])
        grad = math.ceil(((grad+12)%12) / 2)
        name = '{}{}'.format(num, listaMod[grad])
        prog.append([creatChord(name, note), grad])
    return prog

In [8]:
def newChordProg(ton, notes):
    """Create a new individual given the tonality and the base notes."""
    chords = selChord(ton, notes)
    for c in chords:
        yield c

In [9]:
def check_interval(chord):
    """Return the number of mistakes in the distance between the notes."""
    res = 0
    if chord[2] - chord[1] > 12 or chord[2]-chord[1] < 0:
        res += 15
    if chord[3] - chord[2] > 12 or chord[3]-chord[2] < 0:
        res += 15
        
    if chord[1] == chord[2] or chord[2] == chord[3]:
        res += 1.4
    return res    

In [10]:
def check_2_chords(ch1, ch2):
    """Return the number of mistakes in the intervals between 2 chords."""
    res = 0
    
    # Check for 5° and 8°
    ite1 = map(lambda x,y: y-x, ch1[:-1], ch1[1:])
    ite2 = map(lambda x,y: y-x, ch2[:-1], ch2[1:])
    for inter1, inter2 in zip(ite1, ite2):
        if inter1 == 7 and inter2 == 7:
            res += 15
        elif inter1 == 0 and inter2 == 0:
            res += 15
        elif inter1 == 12 and inter2 == 12:
            res += 15
    
    # Check for big intervals, just to make it more "human" 
    for note1, note2 in zip(ch1[1:], ch2[1:]):
        if abs(note1-note2) >= 7: # 7 equals 5° interval
            res += .7
    
    return res

In [11]:
def evalNumErr(ton, individual):
    """Evaluation function."""
    res = 0
    for prev, item, nex in neighborhood(individual):
        res += check_interval(item[0])
        if prev == None:
            if item[1] != 0:
                res += 6
            continue
        else:
            if prev[1] in [4, 6] and item[1] in [3, 1]:
                res += 20
            res += check_2_chords(prev[0], item[0])
        if nex == None:
            if item[1] in [1, 2, 3, 4, 5, 6]:
                res += 6
    return (res,)
            

In [12]:
def mutChangeNotes(ton, individual, indpb):
    """Mutant function."""
    new_ind = toolbox.clone(individual)
    for x in range(len(individual[0])):
        if random.random() < indpb:
            
            listaOp = OPTIONS_M if ton[-1] == 'M' else OPTIONS_m
            listaMod = MOD_M if ton[-1] == 'M' else MOD_m
            
            note = individual[x][0][0]
            
            name = note%12
            grad = name-int(ton[:-1])
            grad = math.ceil(((grad+12)%12) / 2)
            num = (listaOp[grad][random.randint(0,len(listaOp[grad])-1)] + name +12) %12
            grad = num-int(ton[:-1])
            grad = math.ceil(((grad+12)%12) / 2)
            name = '{}{}'.format(num, listaMod[grad])
            
            new_ind[x] = [creatChord(name, note), grad]
    
    del new_ind.fitness.values
    return new_ind,

In [13]:
def transform_lilypond(ton, indiv):
    """Take one list of chords and print the it in lilypond notation."""
    note_map = dict()
    if ton[-1] == 'M':
        note_map = {0: 'c',
                    1: 'cis',
                    2: 'd',
                    3: 'dis',
                    4: 'e',
                    5: 'f',
                    6: 'fis',
                    7: 'g',
                    8: 'gis',
                    9: 'a',
                    10:'ais',
                    11:'b'
                    }
    else:
        note_map = {0: 'c',
                    1: 'des',
                    2: 'd',
                    3: 'ees',
                    4: 'e',
                    5: 'f',
                    6: 'ges',
                    7: 'g',
                    8: 'aes',
                    9: 'a',
                    10:'bes',
                    11:'b'
                    }
    voces = [[], [], [], []]
    
    for chord in indiv:
        for note, voce in zip(chord, voces):
            
            octave = (note // 12)-4
            name_lily = note_map[note % 12]
            if octave < 0:
                name_lily += ',' * (octave * -1)
            elif octave > 0:
                name_lily += "'" * octave
            voce.append(name_lily)
      
    print('{}|\n{}|\n{}|\n{}|\n'.format(*(' '.join(voce) for voce in reversed(voces))))

In [14]:
def main(ton):
    pop = toolbox.population(n=400)
    hof = tools.HallOfFame(3)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register('avg', numpy.mean)
    stats.register('std', numpy.std)
    stats.register('min', numpy.min)
    stats.register('max', numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=70, stats=stats, halloffame=hof, verbose=True)
    while min(log.select('min')) > 15:
        pop = toolbox.population(n=400)
        pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.3, ngen=70, stats=stats, halloffame=hof, verbose=True)
        
    for best in hof:
        print([x[0] for x in best])
        
        transform_lilypond(ton, [x[0] for x in best])

In [18]:
if __name__ == '__main__':
    line = input('n[#b] notas ')
    ton, notes = setTon(line)
    print(ton, notes)
    
    # ========================= GA setup =========================
    creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
    creator.create('Individual', list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register('creat_notes', newChordProg, ton, notes)
    toolbox.register('individual', tools.initIterate, creator.Individual,
                     toolbox.creat_notes)
    toolbox.register('population', tools.initRepeat, list, toolbox.individual)

    toolbox.register('evaluate', evalNumErr, ton)
    toolbox.register('mate', tools.cxOnePoint)
    toolbox.register('mutate', mutChangeNotes, ton, indpb=0.4)
    toolbox.register('select', tools.selTournament, tournsize=3)
    # =============================================================
    
    main(ton)
    
    """
    a = toolbox.individual()
    transform_lilypond(x[0] for x in a)
   
    
    print(toolbox.evaluate(a))
    b = toolbox.individual()
    print(b)
    print(toolbox.evaluate(b))
    a,b = toolbox.mate(a,b)
    print(a)
    print(b)
    print("_"*20)
    
    a = toolbox.mutate(a)
    print(a)
    print("-"*20)
    """

n[#b] notas 1# 43 50 48 47 45 43 55 52 50 48 52 50 50 43 45 47 48 47 48 52 50 38 43
7M [43, 50, 48, 47, 45, 43, 55, 52, 50, 48, 52, 50, 50, 43, 45, 47, 48, 47, 48, 52, 50, 38, 43]
gen	nevals	avg    	std    	min  	max  
0  	400   	403.236	57.7385	254.9	544.9
1  	240   	354.54 	45.394 	237.2	458.6
2  	269   	317.573	43.4105	222.4	422.5
3  	258   	283.96 	31.5711	189.6	389  
4  	266   	259.575	27.3612	165.8	372.7
5  	261   	239.237	26.5853	165.8	319.6
6  	256   	219.101	24.0935	133.1	285.9
7  	259   	199.6  	25.4129	113.1	273  
8  	272   	182.703	26.3594	102  	269.1
9  	261   	163.051	25.4588	88.8 	251.7
10 	248   	146.239	23.3065	84.5 	219.7
11 	236   	131.855	21.9296	78.6 	214.2
12 	253   	118.956	22.7091	68.2 	196.7
13 	262   	103.664	19.2341	45.1 	165.5
14 	246   	91.5558	17.5652	48.6 	150.8
15 	264   	82.7298	18.8991	40.2 	180.8
16 	250   	72.4593	15.3757	40.2 	133.9
17 	254   	65.3403	15.4764	28.7 	137.4
18 	257   	57.9143	13.8315	28.7 	127.2
19 	249   	52.2485	12.7062	24.5 	111.5
2

In [16]:
human1 = [[43, 55, 62, 71], 
         [50, 54, 62, 69], 
         [48, 55, 64, 72], 
         [47, 55, 66, 74], 
         
         [45, 57, 64, 72], 
         [43, 59, 67, 74], 
         [55, 60, 64, 72], 
         [52, 60, 67, 72], 
         
         [50, 62, 66, 74], 
         [48, 64, 69, 76], 
         [52, 64, 67, 71], 
         [50, 57, 66, 69], 
         
         [50, 54, 62, 69], 
         [43, 55, 62, 71], 
         [45, 52, 64, 72], 
         [47, 55, 62, 67], 
         
         [48, 55, 64, 72], 
         [47, 55, 62, 71], 
         [48, 55, 64, 67], 
         [52, 55, 64, 71], 
         
         [50, 57, 66, 74], 
         [38, 50, 62, 66], 
         [43, 55, 62, 67]
         ]
transform_lilypond(human1)

human2 = [[50, 57, 66, 74],
          [43, 59, 67, 74],
          [49, 57, 64, 69],
          [50, 57, 66, 69],
          
          [52, 55, 64, 71],
          [45, 57, 64, 73],
          [49, 55, 64, 73],
          [50, 54, 62, 69],
          
          [43, 55, 62, 71],
          [45, 52, 61, 69],
          [50, 54, 62, 69]
          ]
transform_lilypond(human2)

human3 = [[46, 53, 62, 70],
          [50, 53, 62, 69],
          [48, 55, 63, 72],
          [51, 55, 63, 70],
          
          [53, 57, 62, 74],
          [51, 58, 67, 75],
          [48, 57, 65, 77],
          [45, 57, 63, 72],
          
          [46, 53, 62, 70]
          ]
transform_lilypond(human3)


TypeError: transform_lilypond() missing 1 required positional argument: 'indiv'

### Buenos resultados: